<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment3/blob/main/LM_Huggingface_finetune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93293, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 93293 (delta 91), reused 138 (delta 53), pack-reused 93088
Receiving objects: 100% (93293/93293), 85.86 MiB | 5.76 MiB/s, done.
Resolving deltas: 100% (68360/68360), done.


In [ ]:
# Clone should now be in the machine
!ls

Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [ ]:
!nvidia-smi

Thu May  5 07:19:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [ ]:
import os
os.chdir("transformers")
os.chdir("./examples/tensorflow/language-modeling")
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 325 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 94.5 MB/s 
     |████████████████████████████████| 212 kB 78.8 MB/s 
     |████████████████████████████████| 77 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 69.0 MB/s 
     |████████████████████████████████| 136 kB 82.6 MB/s 
     |████████████████████████████████| 127 kB 79.1 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 84.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.

In [ ]:
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [ ]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 26.7 MB 68.2 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [ ]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./tensorflow/language-modeling")

In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-oql1ysyt
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-oql1ysyt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 87.1 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4099889 sha256=e82826c46b830556c25a99de1f9f89ff9d4b7a7919bf3290edce1506975fd621
  Stored in directory: /tmp/pip-ephem-wheel-cache-qpqwz4ib/wheels/fb/1b/91/0fcf504c386d427d65bbaf663eadf8e18cbf9795394ed7050d
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [ ]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('/content/gdrive/MyDrive/dataset_1.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:168346
Evaluation size: 18706


The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [ ]:
!python run_clm.py \
--model_name_or_path distilgpt2 \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 128 \
--save_steps -1 \
--num_train_epochs 20 \
--fp16 \
--output_dir="/content/output"
#--model_type gpt2-medium \

Using custom data configuration default-dcf06268b7da00ab
Extracting data files: 100% 2/2 [00:00<00:00, 843.92it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-dcf06268b7da00ab/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.
100% 2/2 [00:00<00:00, 987.36it/s]
https://huggingface.co/distilgpt2/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0cbmubx0
Downloading: 100% 762/762 [00:00<00:00, 758kB/s]
storing https://huggingface.co/distilgpt2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
creating metadata file for /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a04

In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/output", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
#Print output for each sequence generated above
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

# **Conclusion**
And there you have it, a simple end to end outline on how you can use Colab, Huggingface, and Tensorflow to train and generate new text data using GPT-2. There is a lot of playing around with hyperparameters in the generate phase but given enough tweaking and time you can usually find something that works well with your data and task. I found that even with the larger GPT-2 model and more examples, it could still repeat itself a bit so something you have to generate a large number of sequences before you get a set that you like. Even OpenAI made note of this in their initial results for GPT-2 so if at first it doesnt generate what you want keep trying and playing with the parameters!

One tip I did notice was that if you do not setup your examples with a start token, then you run into the issue of repeated phrases more easily. Given more data that might be less of a problem but I ran into that a lot before putting in the start token of <|title|> in my exmaples. This start token also has the added benefit of giving you a generic starting point in the text generation so that each run is mostly unique from the last run if you do not care about having a specific prompt.